<a href="https://colab.research.google.com/github/GabrielPSMartins/rna_com_keras/blob/main/lab_rna_com_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
print("Olá mundo!")

Olá mundo!
